# Prelude

In [1]:
def input(num):
    return [line.strip() for line in open("data/input%d.txt" % num, "r")]
def map_int(xs):
    return [int(x) for x in xs]

# [Day 5: A Maze of Twisty Trampolines, All Alike](https://adventofcode.com/2017/day/5)

In [30]:
input5 = map_int(input(5))
def solve5(data):
    idx = 0
    size = len(data)
    steps = 0
    while True:
        steps += 1
        next_idx = idx + data[idx]
        if next_idx < 0 or next_idx >= size:
            break
        data[idx] += 1
        idx = next_idx
    return steps
solve5([0,3,0,1,-3])
solve5(input5)

375042

In [31]:
input5 = map_int(input(5))
def solve5b(data):
    idx = 0
    size = len(data)
    steps = 0
    while True:
        steps += 1
        next_idx = idx + data[idx]
        if next_idx < 0 or next_idx >= size:
            break
        if data[idx] >= 3:
            data[idx] -= 1
        else:
            data[idx] += 1
        idx = next_idx
    return steps
#solve5b([0,3,0,1,-3])
solve5b(input5)

28707598

# [Day 6: Memory Reallocation](https://adventofcode.com/2017/day/6)

Part 1

In [36]:
input6 = input(6)
input6 = [int(x.strip()) for x in input6[0].split()]

seen = []
steps = 0
banks = input6
while banks not in seen:
    steps += 1
    seen.append(banks)
    banks = list(banks) # copy?
    idx = banks.index(max(banks))
    to_distribute = banks[idx]
    banks[idx] = 0
    while to_distribute > 0:
        idx += 1
        if idx >= len(banks):
            idx = 0
        banks[idx] += 1
        to_distribute -= 1
steps

4074

Part 2

In [37]:
input6 = input(6)
input6 = [int(x.strip()) for x in input6[0].split()]

def to_dict_key(banks):
    return "-".join([str(x) for x in banks])

seen = {}
steps = 0
banks = input6
while to_dict_key(banks) not in seen:
    seen[to_dict_key(banks)] = steps
    steps += 1
    banks = list(banks) # copy?
    idx = banks.index(max(banks))
    to_distribute = banks[idx]
    banks[idx] = 0
    while to_distribute > 0:
        idx += 1
        if idx >= len(banks):
            idx = 0
        banks[idx] += 1
        to_distribute -= 1
(steps, seen[to_dict_key(banks)], steps - seen[to_dict_key(banks)])

(4074, 1281, 2793)

# [Day 7: Recursive Circus](https://adventofcode.com/2017/day/7)

In [47]:
# Sample input
input7 = """
pbga (66)
xhth (57)
ebii (61)
havc (66)
ktlj (57)
fwft (72) -> ktlj, cntj, xhth
qoyq (66)
padx (45) -> pbga, havc, qoyq
tknk (41) -> ugml, padx, fwft
jptl (61)
ugml (68) -> gyxo, ebii, jptl
gyxo (61)
cntj (57)
"""
input7 = [line.strip() for line in input7.split('\n') if len(line.strip()) > 0]

def to_tuple(input):
    parts = input.split(' ')
    parts[1] = int(parts[1].replace('(','').replace(')',''))
    if len(parts) > 2:
        parts[2] = [p.replace(',','') for p in parts[3:]]
    else:
        parts.append([])
    return (parts[0], parts[1], parts[2])

# Read from file
input7 = input(7)
input7 = list(map(to_tuple, input7))

In [48]:
# Part 1

nodes = {} # dict
def get_node(name):
    if name not in nodes:
        nodes[name] = {'name':name, 'parents':[], 'children':[], 'weight':None} # name, parents, children, weight
    return nodes[name]

for (name, weight, children) in list(input7):
    node = get_node(name)
    node['weight'] = weight
    for child_name in children:
        child_node = get_node(child_name)
        if child_node not in node['children']:
            node['children'].append(child_node)
        if node not in child_node['parents']:
            child_node['parents'].append(node)


[x['name'] for x in nodes.values() if len(x['parents']) == 0]

['qibuqqg']

In [49]:
# Part 2

def sum_tree(node):
    if len(node['children']) == 0:
        return node['weight']
    else:
        return node['weight'] + sum(map(sum_tree, node['children']))

for node in nodes.values():
    node['tree_weight'] = sum_tree(node)

def all_equal(xs):
    if len(xs) == 0:
        return True
    x = xs[0]
    return len([_x for _x in xs if _x != x]) == 0

for node in nodes.values():
    if not all_equal([n['tree_weight'] for n in node['children']]):
        print(node['name'])


qibuqqg
dwggjb
wknuyhc


In [57]:
# wknuyhc has the smallest tree weight, so assume it is the deepest node
# Find the weights of its subtrees
[n['tree_weight'] for n in nodes['wknuyhc']['children']]

[1640, 1640, 1640, 1640, 1647]

In [60]:
# The last subtree is imbalanced. It is 'egbzge'
# 'egbzge' subtree is too heavy by 7, but its subtrees are all equal, so
# subtract 7 from its weight
nodes['egbzge']['weight'] - 7

1079

# [Day 8: I Heard You Like Registers](https://adventofcode.com/2017/day/8)

In [8]:
SAMPLE_INPUT_8 = """
b inc 5 if a > 1
a inc 1 if b < 5
c dec -10 if a >= 1
c inc -20 if c == 10
""".strip().split('\n')

input8 = SAMPLE_INPUT_8
input8 = input(8)

In [12]:
registers = {}
max_register_value = 0

def get_register(name):
    if name not in registers:
        registers[name] = 0
    return registers[name]

def put_register(name,val):
    registers[name] = val
    return val    

def cmp(src, op_type, val):
    if op_type == ">":
        return src > val
    elif op_type == "<":
        return src < val
    elif op_type == "==":
        return src == val
    elif op_type == "!=":
        return src != val
    elif op_type == "<=":
        return src <= val
    elif op_type == ">=":
        return src >= val
    else:
        raise Exception("Unexpected op_typ for cmp: {}".format(op_type))

def op_delta(op_type, val):
    return val if op_type == "inc" else -1*val

def parse(inst):
    parts = inst.split(' ')
    source = get_register(parts[0])
    val = int(parts[2])
    delta = op_delta(parts[1], val)
    condition_register = get_register(parts[4])
    condition_val = int(parts[6])
    condition_result = cmp(condition_register, parts[5], condition_val)
    if condition_result:
        return put_register(parts[0], source + delta)
    return source

    
for inst in input8:
    val = parse(inst)
    max_register_value = max(val, max_register_value)
    

print("max register value at end: {}, max attained: {}".format(max(registers.values()), max_register_value))

max register value at end: 3089, max attained: 5391
